In [1]:
import pandas as pd
import numpy as np
from os import path
import sys
import tensorflow as tf

#plotting
from matplotlib import pyplot as plt

#processing
from scipy.signal import resample

import pandas as pd
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]
HDFStore=pd.HDFStore

from matplotlib import pyplot as plt
%matplotlib inline

#adding local modules
import sys
module_path = path.abspath(path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from preprocess.biodarw_feature_extraction import extract_residues, extract_radio
from loaders.biodarw import load_arquimedes_dataset

In [2]:
!pip install pandas_profiling

     |████████████████████████████████| 262 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 4.0 MB/s eta 0:00:011
     |████████████████████████████████| 102 kB 379 kB/s ta 0:00:01
     |████████████████████████████████| 12.8 MB 14.3 MB/s eta 0:00:01
     |████████████████████████████████| 696 kB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 4.7 MB 34.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 12.1 MB/s eta 0:00:01
     |████████████████████████████████| 812 kB 71.0 MB/s eta 0:00:01
     |████████████████████████████████| 6.9 MB 252 kB/s eta 0:00:01
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27080 sha256=c5e2a381c9bad091c077d6012a20fc96472fb15a1cfeb693f0ccccee16aa8aab
  Stored in directory: /root/.cache/pip/wheels/23/14/6e/4be5bfeeb027f4939a01764b48edd5996acf574b0913fe5243
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295188 sha256=9af5801c340bf711d1feb4aa50d0

In [4]:
import pandas_profiling as pdpf

In [5]:
features=['x','y','pen_up','pressure']
cols=[0, 1, 3, 6]
doc_path = "/data/elekin/doc"
meta_df=pd.read_csv(path.join(doc_path,"metadata-202106-v1.csv"),index_col=0)
x_train = []
y_train = []

In [6]:
# pdpf.ProfileReport(meta_df).to_file(doc_path+"/spiral-meta-profiling.html")


In [7]:
# Creating a data structure with subjects time series and tremor levels
 
X = []
y = []

for path, level in zip(meta_df.abs_path, meta_df.level):
    df = pd.read_csv(path, sep="\s+",header=None,names=features,skiprows=1,usecols=[0,1,3,6])
    X.append(resample(df.values.astype('int16'), 4096))
    y.append(level)

In [12]:
df

,x,y,pen_up,pressure
0,3880,5330,1,1229
1,3870,5330,1,1229
2,3870,5330,0,0
3,5310,7740,0,0
4,5306,7747,0,0
...,...,...,...,...
1560,5637,8764,1,1832
1561,5637,8764,1,1794
1562,5637,8764,1,1688
1563,5637,8764,1,1409


In [6]:
with open('feat.npy', 'wb') as temp_feat:
    np.save(temp_feat, np.array(X, dtype=np.float32), allow_pickle=True)
with open('labels.npy', 'wb') as temp_labels:
    np.save(temp_labels, np.array(y, dtype=np.int8), allow_pickle=True)

In [7]:
with open('labels.npy', 'rb') as temp_labels:
    y_train = np.load(temp_labels, allow_pickle=True)
with open('feat.npy', 'rb') as temp_feat:
    X_train = np.load(temp_feat, allow_pickle=True)

In [8]:
X_train.shape

(53, 4096, 4)

In [9]:
y_train.shape

(53,)

In [10]:
dataset = tf.data.Dataset.from_tensor_slices((X_train, tf.one_hot(y_train, 3)))

In [11]:
tf.data.experimental.save(dataset, "./ethw.tfdata")

In [12]:
len(dataset)

53

In [13]:
for feat, label in tf.data.experimental.load("./ethw.tfdata").take(1):
    print(feat.shape)
    

(4096, 4)
